In [1]:
!pip3 install --user pandas_datareader==0.7.0
!pip3 install --user yfinance
!pip3 install --user matplotlib
!pip3 install --user seaborn
!pip3 install --user networkx
!pip3 install --user qiskit
!pip3 install --user launchpadlib
!pip3 install --user cryptography
!pip3 install --user pandas
!pip3 install --user Quandl
!pip3 install --user intrinio-sdk
!pip3 install --user ipynb

from ipynb.fs.full.APIKey import *

import quandl

from IPython.core.debugger import set_trace
import yfinance as yf
import os, pickle
import pandas as pd
import numpy as np

### Import plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm, laplace, johnsonsu
from scipy import stats


#Remove warnings
import warnings


import scipy.sparse as sp
import networkx as nx

import scipy.optimize
import collections
import math
import time
import itertools
from scipy.linalg import expm, sinm, cosm, logm
from qiskit import *

sns.set(style="darkgrid")
warnings.filterwarnings('ignore')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [2]:
from __future__ import print_function
import time
import intrinio_sdk
from intrinio_sdk.rest import ApiException
from pprint import pprint

#Get your API from https://account.intrinio.com/
intrinio_sdk.ApiClient().configuration.api_key['api_key'] = personal_API

security_api = intrinio_sdk.SecurityApi()
stock_exchange_api = intrinio_sdk.StockExchangeApi()

identifier = 'USCOMP' # str | A Stock Exchange identifier (MIC or Intrinio ID)
date = '2018-08-14' # date | The date for which to return prices (optional)
page_size = 100 # int | The number of results to return (optional) (default to 100)
next_page = '' # str | Gets the next page of data from a previous API call (optional)

try:
  api_response = stock_exchange_api.get_stock_exchange_prices(identifier, date=date, page_size=page_size, next_page=next_page)
  pprint(api_response)
except ApiException as e:
  print("Exception when calling StockExchangeApi->get_stock_exchange_prices: %s\r\n" % e)
    
    

{'next_page': None,
 'stock_exchange': {'acronym': 'USCOMP',
                    'city': None,
                    'country': 'UNITED STATES OF AMERICA',
                    'country_code': 'US',
                    'first_stock_price_date': datetime.date(1962, 1, 2),
                    'id': 'sxg_ozMr9y',
                    'last_stock_price_date': datetime.date(2019, 8, 19),
                    'mic': 'USCOMP',
                    'name': 'US COMPOSITE',
                    'website': None},
 'stock_prices': [{'adj_close': 205.926638699597,
                   'adj_high': 206.721873871691,
                   'adj_low': 204.463798691671,
                   'adj_open': 206.324256285644,
                   'adj_volume': 20748010.0,
                   'close': 209.75,
                   'date': datetime.date(2018, 8, 14),
                   'frequency': 'daily',
                   'high': 210.56,
                   'intraperiod': False,
                   'low': 208.26,
                

In [3]:
df = pd.read_csv('2018tradesTicks.csv',names=['UTCDate_UTCTime', 'Price', 'Quantity', 'isBuy', 'isSell'])
df

FileNotFoundError: [Errno 2] File b'2018tradesTicks.csv' does not exist: b'2018tradesTicks.csv'

In [ ]:
df_precios_unique=df['Price'].unique()


precios_unique = df.pivot_table(index=['Price'], aggfunc='size')

df_precios = pd.DataFrame()
df_precios['Price']=df_precios_unique
df_precios['Count']=precios_unique.tolist()

sns.distplot(precios_unique);
print(df_precios)    


In [ ]:
import statistics
partition=64

df_split = np.array_split(df_precios, partition)

precios_suma=df_precios['Count'].sum()
print(precios_suma)

indice=0
arr_vector=[]
arr_median=[]
for dataframe in df_split:
    arr_temp_values=[]
    arr_temp_price=[]
    arr_median_temp=[]
    sumator=0
    for index, row in df_split[indice].iterrows():
        value=row['Count']/precios_suma
        price=row['Price']
        arr_temp_values.append(value)
        arr_temp_price.append(price)
        arr_median_temp.append(row['Price'])
        sumator=sumator+value
    median=statistics.median(arr_median_temp)
    arr_median.append(median)
    arr_vector.append(sumator)   
    minimum=[]   
    maximum=[]
    suma=[]
    peak=[]
    minimum.append(df_split[indice]['Count'].min())
    maximum.append(df_split[indice]['Count'].max())
    suma.append(df_split[indice]['Count'].sum())
    peak.append(str((maximum[0]-minimum[0])/suma[0]))
    
    
    df_temp_peak=pd.DataFrame()
    df_temp_peak['MIN']=minimum
    df_temp_peak['MAX']=maximum
    df_temp_peak['SUM']=suma
    df_temp_peak['PEAK']=peak
    
    df_temp_peak.to_csv('df_temp_peak'+str(indice)+'.csv', sep=';')
        
    df_temp=pd.DataFrame()
    df_temp['Prices']=arr_temp_price
    df_temp['Percent']=arr_temp_values
    df_temp.to_csv('df_temp_'+str(indice)+'.csv', sep=';')
    indice=indice+1
    
print(arr_median)
print(arr_vector)


In [ ]:
# Este bloque lleva un ansatz de los parámetros del Hamiltoniano a un set de parámetros que minimiza la "distancia"
# entre las distribuciones de probabilidad objetivo, y simulada.

N=6
sx = sp.csr_matrix([[0.0,1.0],[1.0,0.0]])
sz = sp.csr_matrix([[1.0,0.0],[0.0,-1.0]])
sy = -1j * (sz @ sx)

def generate_operators(op, N):
    return [sp.kron(sp.eye(2**i),
                    sp.kron(op, sp.eye(2**(N-i-1))))
            for i in range(N)]

sx = generate_operators(sx, N)
sy = generate_operators(sy, N)
sz = generate_operators(sz, N)
p= np.random.normal(0,3,int(N*(N-1)/2+2*N))

        
                
def cost(args):
        
    Psi=np.ones(2**N)*1/(2**(N/2))     
    Index=np.zeros(N)
    Index[0:2]=3
    a=list(itertools.permutations(Index))
    a=list(dict.fromkeys(a))
    H=0
    for i in range(int(N*(N-1)/2)):
            
        s=1
        for j in a[i]:
                
            if j==0:
                s1=sp.csr_matrix([[1.0,0.0],[0.0,1.0]])
            if j==3:
                s1=sp.csr_matrix([[1.0,0.0],[0.0,-1.0]])                    
            s=sp.kron(s,s1)
                
        H=H+args[i]*s
        
    for j in range(N):
            
        H=H+sz[j]*args[j+int(N*(N-1)/2)]+sx[j]*args[j+int(N*(N-1)/2+N)]
        
    Yexp=np.conj(expm(H)@Psi)*(expm(H)@Psi)
    Yteo=arr_vector
    cost=0
    for i in range(2**N):
            
        cost=Yteo[i]*np.log(Yexp[i])
             
    return cost
             
opt = scipy.optimize.minimize(cost, p)
opt['x']

In [ ]:
t = 10.
time_steps = 20

In [ ]:
# Mediante puertas CNOT y rotaciones en x y en z, construimos un estado superposición final cuyas amplitudes de
# probabilidad se parecen lo máximo posible a las de la distribución objetivo construida con los datos históricos.

circ = QuantumCircuit(N,N)

# Puertas lógicas que simulan la interacción del Hamiltoniano de Ising 

Index      = np.zeros(N)
Index[0:2] = 1.
a = list(itertools.permutations(Index))
a = list(dict.fromkeys(a))

optimized_p = {}

#Creamos un estado máximamente entrelazado

for i in range(N):
    
    circ.h(i)
        
circ.barrier()

for l in range(time_steps) :

    #Simulamos la evolución del estado con puertas CNOT rotaciones en z y puertas x y z.
    
    for i in range(len(a)):
      
        optimized_p[a[i]] = opt['x'][i]
        s = []
        k=-1
        
        for j in a[i] :
            
            k = k + 1
            
            if j==1 :
                
                s.append(k) 
         
        circ.cx(s[0],s[1])
        circ.rz((t/time_steps)*optimized_p[a[i]],s[1])
        circ.cx(s[0],s[1])

    
    # Puertas lógicas que simulan el término onsite y  del Hamiltoniano de Ising 

    for i in range(N) :
    
        circ.rz((t/time_steps)*opt['x'][int(N*(N-1)/2)+i],i)
        circ.rx((t/time_steps)*opt['x'][int(N*(N-1)/2+N)+i],i)
        
    circ.barrier()
    
for i in range(N):
        
    circ.measure(i,i)
    

In [ ]:
opt

In [ ]:
circ.draw(output='mpl')

In [ ]:
backend_sim = Aer.get_backend('qasm_simulator')
NN=1 # One shot experiment

temporal_series = []

for i in range(1000):

    job = execute(circ, backend = Aer.get_backend('qasm_simulator'), shots=NN)
    result = job.result()
    counts = result.get_counts(circ)
    
    temporal_series.append(list(counts.keys())[0])
    

counts={}

#Escribe el diccionario con las medidas de los estados, tras hacer el experimento 1000 veces en este caso.

for i in temporal_series:

    if i in counts.keys() : 
        
        counts[i] = counts[i] + 1.
        
    else :
        
        counts[i] = 1.
        
# Número de veces que se mide cada estados
print(counts)
    
for i in counts:
    
    counts[i] = counts[i]/1000


prices_series=[]

# Identifica las etiquetas de los estados con los precios
l=-1

for i in temporal_series:
    l=l+1
    suma = 0
    k=0
    
    for j in i:
        
        k = k + 1    
        suma = suma + int(j)*2**(len(i)-k)
        
    prices_series.append(arr_median[suma])


#Plot the syntetic temporal price series
pd_price_series=pd.DataFrame()
pd_price_series['prices_series']=prices_series
pd_price_series

#sns.lineplot(x=range(1,len(prices_series)+1), y=prices_series,
#             data=pd_price_series)
    
#for j in counts.keys():
#   resultados_porcentuales[j]=round((resultados[j]/S)**(0.5),2) 
#   c_porcentuales[j]=round((counts[j]/NN),2)
#   print(c_porcentuales)

temporal_series


In [ ]:
# Serie temporal para los primeros 1000 precios históricos de 2018.

len(temporal_series)


sns.lineplot(x=range(0,1000), y=df[:1000]['Price'],
             data=df[:1000])

df_precios

In [ ]:
# Grafica la distribución de probabilidad histórica

import seaborn as sns
sns.set(style="darkgrid")

arr_temporal_series_keys=[]
arr_temporal_series_count=[]
for value in counts.keys():
    arr_temporal_series_keys.append(value)

for count in arr_temporal_series_keys:
    arr_temporal_series_count.append(counts[count])

counts_dataframe=pd.DataFrame()
counts_dataframe['Keys']=arr_temporal_series_keys
counts_dataframe['Count']=arr_temporal_series_count

sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.lineplot(x=counts_dataframe['Keys'], y=counts_dataframe['Count'],
             data=counts_dataframe)

In [ ]:
# Grafica la distribución de probabilidad sintética

mean_vector_pd=pd.DataFrame()
mean_vector_pd['Median']=arr_median
mean_vector_pd['Proba']=arr_vector

sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.lineplot(x=mean_vector_pd['Median'], y=mean_vector_pd['Proba'],
             data=mean_vector_pd)

In [ ]:
# Guarda en un .csv las claves obtenidas en las medidas y sus conteos.

import pandas as pd
pd_hess_inv=pd.DataFrame()
pd_hess_inv['counts.keys']=counts.keys()
pd_hess_inv['count.values']=counts.values()
pd_hess_inv

pd_hess_inv.to_csv('df_pd_hess_inv.csv', sep=';')

pd_temporal_series=pd.DataFrame()
pd_temporal_series['Temporal_series']=temporal_series

pd_temporal_series.to_csv('df_pd_temporal_series.csv', sep=';')

In [ ]:
# Grafica la serie temporal sintética

pd_prices_series=pd.DataFrame()
pd_prices_series['Prices']=prices_series
print(len(prices_series))
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.lineplot(x=range(0,1000), y=prices_series,
             data=pd_prices_series)